In [153]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os

In [154]:
jumps = pd.read_csv('test/Markov.txt', sep=' ', header=None)
jumps.columns = ['ci', 't_curr', 't_next', 'px', 'py', 'pz']

jumps['t_prev'] = jumps.groupby('ci')['t_curr'].shift(1)
jumps['px_prev'] = jumps.groupby('ci')['px'].shift(1)
jumps['py_prev'] = jumps.groupby('ci')['py'].shift(1)
jumps['pz_prev'] = jumps.groupby('ci')['pz'].shift(1)

jumps = jumps[['t_prev', 't_curr', 't_next', 'px', 'py', 'pz', 'px_prev', 'py_prev', 'pz_prev']]
jumps.dropna(inplace=True)
jumps = jumps.astype({'t_prev': 'int32', 't_curr': 'int32', 't_next': 'int32'})
jumps

,t_prev,t_curr,t_next,px,py,pz,px_prev,py_prev,pz_prev
14,22822,22824,22822,-0.173278,0.593864,0.785685,-0.845382,0.528670,-0.076397
17,42957,42954,42957,0.620560,0.783458,0.033144,0.566898,-0.458318,0.684523
22,33944,33943,33944,-0.857558,-0.326207,0.397723,-0.952018,-0.302227,-0.048165
24,42954,42957,42954,-0.863366,0.343955,0.369181,0.620560,0.783458,0.033144
38,60909,60910,60909,-0.819193,0.534514,-0.207888,0.477167,-0.525350,-0.704499
...,...,...,...,...,...,...,...,...,...
14485,94347,147931,147930,-0.800684,-0.598769,0.019521,-0.800684,-0.598769,0.019521
14486,94705,94706,94705,0.130832,-0.604047,0.786136,-0.903331,0.349608,0.248531
14487,88084,88085,88084,-0.964489,0.185228,0.188285,0.753496,-0.399304,-0.522302
14488,4369,65560,4369,-0.140546,-0.109370,-0.984015,-0.268848,0.890343,-0.367436


In [155]:
jumps['corr'] = (jumps['px']*jumps['px_prev'] + jumps['py']*jumps['py_prev'] + jumps['pz']*jumps['pz_prev'])
jumps['corr'].mean()

-0.22213926525739333

In [156]:
channel = pd.read_csv('test/1000_42_area.txt')
channel['area_sum'] = channel.groupby('i')['area'].transform('sum')
channel['P'] = channel['area'] / channel['area_sum']
channel

,i,j,area,area_sum,P
0,525,4302,1.346497,3.461245,0.389021
1,525,4379,1.201940,3.461245,0.347256
2,525,1735,0.912809,3.461245,0.263723
3,587,1636,1.262791,3.391623,0.372326
4,587,4367,0.116779,3.391623,0.034432
...,...,...,...,...,...
1576,173376,173378,0.227806,0.227806,1.000000
1577,173377,173383,0.243089,0.757532,0.320897
1578,173377,173378,0.514443,0.757532,0.679103
1579,177284,177286,0.443462,0.443462,1.000000


In [ ]:
jumps = jumps.merge(channel[['i', 'j', 'P']], left_on=['t_curr', 't_prev'], right_on=['i','j'], how='left')
jumps = jumps[['t_prev', 't_curr', 't_next', 'P']]
jumps.dropna(inplace=True)
jumps

,t_prev,t_curr,t_next,P
1,42957,42954,42957,0.490727
2,33944,33943,33944,0.319804
5,23187,21508,23187,0.265405
8,77328,60921,77328,0.022437
10,77312,77311,77312,1.000000
...,...,...,...,...
13809,36874,2898,36874,0.430968
13810,60928,36919,60928,1.000000
13811,45843,45841,45843,0.270276
13815,23175,10322,23175,0.862638


In [158]:
jumps[jumps['t_next'] == jumps['t_prev']]

,t_prev,t_curr,t_next,P
1,42957,42954,42957,0.490727
2,33944,33943,33944,0.319804
5,23187,21508,23187,0.265405
8,77328,60921,77328,0.022437
10,77312,77311,77312,1.000000
...,...,...,...,...
13809,36874,2898,36874,0.430968
13810,60928,36919,60928,1.000000
13811,45843,45841,45843,0.270276
13815,23175,10322,23175,0.862638


In [159]:
jumps['P'].mean()

0.6526351327017961

In [160]:
print("Expected Next == Prev: ", jumps['P'].mean())
print('Observed Next == Prev: ', len(jumps[jumps['t_next'] == jumps['t_prev']])/len(jumps))

Expected Next == Prev:  0.6526351327017961
Observed Next == Prev:  0.8500146756677429
